https://llama-cpp-python.readthedocs.io/en/latest/

In [7]:
from llama_cpp import Llama
max_tokens = 32000
# llama_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-7b-chat.Q5_K_M.gguf", chat_format="llama-2")
model_file_path = "/Users/jmcaffee/llm_models/Llama-3-8B-Instruct-32k-v0.1.Q4_K_M.gguf"
llm = Llama(model_path=model_file_path, chat_format="llama-3", n_ctx=max_tokens)
llm.verbose = False # don't print a bunch of stuff
# llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/llama-2-13b-chat.Q5_K_M.gguf", chat_format="llama-2", n_ctx=2048) # n_ctx=2048
# function_llm = Llama(model_path="/Volumes/4Tera_SSD_2022/llm_models/functionary-7b-v1.Q5_K.gguf", chat_format="llama-2") # n_ctx=2048

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/jmcaffee/llm_models/Llama-3-8B-Instruct-32k-v0.1.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--MaziyarPanahi--Llama-3-8B-Ins...
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32000
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model

In [8]:
import time
import json

def prompt(question):
    output = llm(
        f"Q: {question} A:", # Prompt
        # max_tokens=512,
        max_tokens=max_tokens,
        stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
        echo=False, # Echo the prompt back in the output
    )
    answer = output['choices'][0]['text']
    print(f"Q: {question} \nA:{answer}")
    return answer

def prompt2(question):
    start_time_seconds = time.time()
    output = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ]
    )
    elapsed_time_seconds = time.time() - start_time_seconds
    # print(output)
    answer = output['choices'][0]['message']['content']
    print(f'answer generated in {elapsed_time_seconds} seconds')
    print(f"Q: {question} \nA:{answer}")
    return answer

def stream_prompt(question):
    start_time_seconds = time.time()
    stream = llm.create_chat_completion(
        max_tokens=max_tokens,
        messages = [
            {
                "role": "system",
                "content": "You are an assistant who perfectly responds to questions with as many details as possible, without making up facts."
            },
            {
                "role": "user",
                "content": question
            }
        ],
        stream = True,
    )


    for output in stream:
        choices = output['choices']
        choice = choices[0]
        delta = choice['delta']
        if 'content' in delta:
            print(delta['content'], end='')
    print()

    elapsed_time_seconds = time.time() - start_time_seconds
    print(f'answer generated in {elapsed_time_seconds} seconds')

In [9]:
stream_prompt("Who was Juan de Leon?")

Juan Sebastián Elcano (1476-1526) was a Spanish explorer and navigator who is best known for being the first person to circumnavigate the globe in a single expedition. He was born in Getaria, Spain, and his full name was Juan Sebastián Elcano y Hurtado de M

KeyboardInterrupt: 

In [11]:
with open('../parse_openapi/plaid-openai-parse-results/plaid_accounts_get.html', 'r') as file:
    large_text = file.read()
    
stream_prompt(f"""
Using the documentation below, what endpoint do I call to get the account balance?
{large_text}
""")    

According to the documentation, you can get the account balance by calling the `/accounts/balance/get` endpoint. This endpoint returns the current and available balances for an account, as well as other information about the account's type, subtype, and verification status.
answer generated in 62.31879496574402 seconds
